In [1]:
from lib.logging import logger

In [2]:
from pipeline import SettlementPipeline
import settle
from transaction_parser import StringTransactionInputParser

allTransactions = '''
group A B C D E F
A 40 D
A 10 E
B 30 F
B 30 E
C 70 E
'''

pipeline = SettlementPipeline(inputParserKlass=StringTransactionInputParser, settlerKalss=settle.Settler, settlerKwargs=dict(nParticipantsThreshold=8))
pipeline.getSettlementTransactions(allTransactions)

2026-02-08 23:47:17.344India Standard Time 6000 22716 INFO Settling with config: Settler(nParticipantsThreshold=8)
2026-02-08 23:47:17.349India Standard Time 6000 22716 INFO Using OptimialTransactionSettler settlement strategy for 6 participants
2026-02-08 23:47:17.349India Standard Time 6000 22716 INFO Number of transactions: 4
2026-02-08 23:47:17.349India Standard Time 6000 22716 INFO D has to pay C 40.0
2026-02-08 23:47:17.353India Standard Time 6000 22716 INFO E has to pay B 60.0
2026-02-08 23:47:17.354India Standard Time 6000 22716 INFO E has to pay A 50.0
2026-02-08 23:47:17.354India Standard Time 6000 22716 INFO F has to pay C 30.0


In [3]:
pipeline = SettlementPipeline(inputParserKlass=StringTransactionInputParser, settlerKalss=settle.Settler, settlerKwargs=dict(nParticipantsThreshold=5))
pipeline.getSettlementTransactions(allTransactions)

2026-02-08 23:47:17.381India Standard Time 6000 22716 INFO Settling with config: Settler(nParticipantsThreshold=5)
2026-02-08 23:47:17.381India Standard Time 6000 22716 INFO Using GreedyTransactionSettler settlement strategy for 6 participants
2026-02-08 23:47:17.387India Standard Time 6000 22716 INFO Number of transactions: 5
2026-02-08 23:47:17.389India Standard Time 6000 22716 INFO D has to pay B 20.0
2026-02-08 23:47:17.389India Standard Time 6000 22716 INFO D has to pay A 20.0
2026-02-08 23:47:17.389India Standard Time 6000 22716 INFO E has to pay C 70.0
2026-02-08 23:47:17.389India Standard Time 6000 22716 INFO E has to pay B 40.0
2026-02-08 23:47:17.420India Standard Time 6000 22716 INFO F has to pay A 30.0


In [4]:
allTransactions = '''
group A B C D E F
A 40 D
A 10 E
B 30 F
B 30 E,
C 70 E
'''
try:
    pipeline.getSettlementTransactions(allTransactions)
except ValueError as ve:
    logger.exception(f'value error: {ve}')

2026-02-08 23:47:17.455India Standard Time 6000 22716 ERROR value error: invalid chars present in the string
Traceback (most recent call last):
  File "C:\Users\suraj\AppData\Local\Temp\ipykernel_6000\304248226.py", line 10, in <module>
    pipeline.getSettlementTransactions(allTransactions)
  File "c:\Users\suraj\projects\cospend\pipeline.py", line 24, in getSettlementTransactions
    self._validate(inputTransactions)
  File "c:\Users\suraj\projects\cospend\pipeline.py", line 17, in _validate
    self.inputParserKlass.validate(inputString)
  File "c:\Users\suraj\projects\cospend\transaction_parser.py", line 42, in validate
    cls.validateChars(inputString)
  File "c:\Users\suraj\projects\cospend\transaction_parser.py", line 37, in validateChars
    raise ValueError('invalid chars present in the string')
ValueError: invalid chars present in the string


In [5]:
allTransactions = '''
group A B C D E F
A 40 D
A 10 E
B 30 F
B 30 E
C 70 E
'''
pipeline = SettlementPipeline(inputParserKlass=StringTransactionInputParser, settlerKalss=settle.SmartTransactionSettler, settlerKwargs=dict(depth=1))
pipeline.getSettlementTransactions(allTransactions)

2026-02-08 23:47:17.479India Standard Time 6000 22716 INFO Number of transactions: 4
2026-02-08 23:47:17.479India Standard Time 6000 22716 INFO D has to pay C 40.0
2026-02-08 23:47:17.479India Standard Time 6000 22716 INFO E has to pay B 60.0
2026-02-08 23:47:17.479India Standard Time 6000 22716 INFO E has to pay A 50.0
2026-02-08 23:47:17.479India Standard Time 6000 22716 INFO F has to pay C 30.0


In [8]:
sr = pipeline.getSettlementTransactions(allTransactions)
for t in sr:
    print(t)
tt = []
tt.extend(sr)
tt

2026-02-08 23:46:30.804India Standard Time 4820 17040 INFO Number of transactions: 4
2026-02-08 23:46:30.804India Standard Time 4820 17040 INFO D has to pay C 40.0
2026-02-08 23:46:30.804India Standard Time 4820 17040 INFO E has to pay B 60.0
2026-02-08 23:46:30.804India Standard Time 4820 17040 INFO E has to pay A 50.0
2026-02-08 23:46:30.810India Standard Time 4820 17040 INFO F has to pay C 30.0


('d', 'c', np.float64(40.0))
('e', 'b', np.float64(60.0))
('e', 'a', np.float64(50.0))
('f', 'c', np.float64(30.0))


[('d', 'c', np.float64(40.0)),
 ('e', 'b', np.float64(60.0)),
 ('e', 'a', np.float64(50.0)),
 ('f', 'c', np.float64(30.0))]

In [7]:
import datetime

class SpendingRecord:
    def __init__(self, amount:float|int, spender:str, members:tuple|None = None, note:str|None = ''):
            self.entryTime = datetime.datetime.now()
            self.amount = amount
            self.spender = spender
            self.members = members or []
            self.note = note

    def __repr__(self):
        return f'created @ {self.entryTime} {self.__class__.__name__}(amount={self.amount}, spender={self.spender}, members={self.members}, note={self.note or ""})'

    def logRecord(self):
        return f'{self.spender} {self.amount} {' '.join(self.members)}'
    
    def __str__(self):
        res = f'{self.entryTime} {self.spender} spent {self.amount}'
        if self.members:
            res = f'{res} on {self.members}'
        if self.note:
            res = f'{res} for {self.note}'
        return res

class SpendingTransactionRecorder:

    BASE_DIR = '/transcations/'

    def __init__(self, groupName:str, members:set):
        self.groupName = groupName
        self.members = members
        self._path = f'{self.BASE_DIR}{self.groupName}.txt'
        self._transactions = []

    def addRecord(self, amount:float|int, spender:str, members:str|set|None = None, note:str|None = ''):
        if isinstance(members, str):
            if members.startswith('!'):
                members = set(m for m in self.members if m!=members[1:])
        members = set(members)
        if members.difference(self.members):
            raise ValueError(f'Unknown member(s) present in the entry: {members.difference(self.members)}')
        self._transactions.append(SpendingRecord(amount, spender, members, note))

    def showRecords(self):
        for entry in self._transactions:
            logger.info(entry)

    def logRecords(self):
        for entry in self._transactions:
            logger.info(entry.logRecord())


In [70]:
import datetime

class SpendingRecord:
    def __init__(self, amount:float|int, spender:str, members:tuple|None = None, note:str|None = ''):
            self.entryTime = datetime.datetime.now()
            self.amount = amount
            self.spender = spender
            self.members = members or []
            self.note = note

    def __repr__(self):
        return f'created @ {self.entryTime} {self.__class__.__name__}(amount={self.amount}, spender={self.spender}, members={self.members}, note={self.note or ""})'

    def logRecord(self):
        return f'{self.spender} {self.amount} {' '.join(self.members)}'
    
    def __str__(self):
        res = f'{self.entryTime} {self.spender} spent {self.amount}'
        if self.members:
            res = f'{res} on {self.members}'
        if self.note:
            res = f'{res} for {self.note}'
        return res

class SpendingTransactionRecorder:

    BASE_DIR = '/transcations/'

    def __init__(self, groupName:str, members:set):
        self.groupName = groupName
        self.members = members
        self._path = f'{self.BASE_DIR}{self.groupName}.txt'
        self._transactions = []
        self._flowMap = {m:0 for m in self.members}

    def addRecord(self, amount:float|int, spender:str, members:str|set|None = None, note:str|None = ''):
        if isinstance(members, str):
            if members.startswith('!'):
                members = set(m for m in self.members if m!=members[1:])
        members = set(members)
        if members.difference(self.members):
            raise ValueError(f'Unknown member(s) present in the entry: {members.difference(self.members)}')
        self._transactions.append(SpendingRecord(amount, spender, members, note))

        self._flowMap[spender]+=float(amount)
        for m in members:
            self._flowMap[m]-=float(amount)/len(members)

    def showFlowMap(self):
        return self._flowMap

    def showRecords(self):
        for entry in self._transactions:
            logger.info(entry)

    def logRecords(self):
        for entry in self._transactions:
            logger.info(entry.logRecord())


In [76]:
import time
import random
randomsleep = lambda: time.sleep(random.randint(1, 2))
recorder = SpendingTransactionRecorder(groupName='testgroup', members=set(['A', 'B', 'C', 'D', 'E', 'F']))
recorder.showRecords()
recorder.addRecord(40, 'A', ['D'], 'coffee')
randomsleep()
recorder.addRecord(10, 'A', ['E'], 'water')
#randomsleep()
recorder.addRecord(30, 'B', ['F'], 'tea')
#randomsleep()
recorder.addRecord(30, 'B', ['E'])
#randomsleep()
recorder.addRecord(70, 'C', 'F', 'snack')
randomsleep()
recorder.showRecords()

2026-02-08 22:43:41.345India Standard Time 19224 10372 INFO 2026-02-08 22:43:37.336930 A spent 40 on {'D'} for coffee
2026-02-08 22:43:41.345India Standard Time 19224 10372 INFO 2026-02-08 22:43:39.341212 A spent 10 on {'E'} for water
2026-02-08 22:43:41.345India Standard Time 19224 10372 INFO 2026-02-08 22:43:39.341212 B spent 30 on {'F'} for tea
2026-02-08 22:43:41.349India Standard Time 19224 10372 INFO 2026-02-08 22:43:39.341212 B spent 30 on {'E'}
2026-02-08 22:43:41.349India Standard Time 19224 10372 INFO 2026-02-08 22:43:39.341212 C spent 70 on {'F'} for snack


In [77]:
recorder.logRecords()

2026-02-08 22:43:43.595India Standard Time 19224 10372 INFO A 40 D
2026-02-08 22:43:43.600India Standard Time 19224 10372 INFO A 10 E
2026-02-08 22:43:43.600India Standard Time 19224 10372 INFO B 30 F
2026-02-08 22:43:43.600India Standard Time 19224 10372 INFO B 30 E
2026-02-08 22:43:43.600India Standard Time 19224 10372 INFO C 70 F


In [78]:
for t in recorder._transactions:
    print(repr(t))

created @ 2026-02-08 22:43:37.336930 SpendingRecord(amount=40, spender=A, members={'D'}, note=coffee)
created @ 2026-02-08 22:43:39.341212 SpendingRecord(amount=10, spender=A, members={'E'}, note=water)
created @ 2026-02-08 22:43:39.341212 SpendingRecord(amount=30, spender=B, members={'F'}, note=tea)
created @ 2026-02-08 22:43:39.341212 SpendingRecord(amount=30, spender=B, members={'E'}, note=)
created @ 2026-02-08 22:43:39.341212 SpendingRecord(amount=70, spender=C, members={'F'}, note=snack)


In [79]:
recorder.showFlowMap()

{'F': -100.0, 'C': 70.0, 'B': 60.0, 'E': -40.0, 'D': -40.0, 'A': 50.0}